# GLM-4-Plus-VideoCall Audio Call

**This tutorial is available in English and is attached below the Chinese explanation**

该脚本使用 GLM-4-Plus-VideoCall 输入音频文件，并通过 WebSocket 连接将其发送到服务器转录音频中的人声内容。

This script uses GLM-4-Plus-VideoCall to input an audio file and send it over a WebSocket connection to a server for a transcription of the human voice in the audio. 

## 1. Import and Encode Audio File

首先导入音频处理、编码和 WebSocket 通信所需的库。然后读取 WAV 文件并将其编码为 Base64。WAV 文件以未压缩格式存储音频数据，因此适合编码和发送。

Start by importing the required libraries for audio handling, encoding, and WebSocket communication. Then read a WAV file and encode it to Base64. WAV files store audio data in an uncompressed format, making it suitable for encoding and sending. 

In [1]:
import websockets
import json
import time
import base64
import wave

def read_and_encode_audio(file_path):
    with wave.open(file_path, 'rb') as audio_file:

            channels = audio_file.getnchannels()
            sample_width = audio_file.getsampwidth()
            frame_rate = audio_file.getframerate()

            print(f"Channels: {channels}, Sample Width: {sample_width}, Frame Rate: {frame_rate}")

            audio_data = audio_file.readframes(audio_file.getnframes())

            base64_encoded_audio = base64.b64encode(audio_data).decode('utf-8')

            return base64_encoded_audio

def save_to_file(data, output_path):
    with open(output_path, 'w') as file:
        file.write(data)
        print(f"Base64 编码的音频数据已保存到文件：{output_path}")

file_path = "vision/data/lecture.wav"
output_file_path = "vision/data/audio_base64_output.txt"

base64_audio = read_and_encode_audio(file_path)

if base64_audio:
    save_to_file(base64_audio, output_file_path)


Channels: 2, Sample Width: 2, Frame Rate: 44100
Base64 编码的音频数据已保存到文件：/Users/lezhiliu/Desktop/glm-cookbook/vision/data/audio_base64_output.txt


## 2. Save the Base64 String
进行编码后，将 Base64 字符串保存为外部的 txt 文件。

After encoding, the Base64 string is saved as an external txt file.

In [2]:
def read_base64_file(file_path):
    with open(file_path, 'r') as file:
            return file.read()


## 3. Establishing a WebSocket Connection for Audio Transcription
现在，通过 connect_websocket 函数，我们建立了一个与指定 URI 的 WebSocket 连接，该连接带有授权标头，并使用承载令牌进行身份验证。连接后，它首先会收到服务器的初始响应。然后，它从指定路径读取一个 Base64 编码的音频文件，并构建一个包含各种参数的信息：一个客户端时间戳、一个提示（要求服务器转录音频）和一个控制部分，其中指定了文本响应格式、基于服务器的语音活动检测（VAD）、PCM 音频编码、语音类型和速比。发送 JSON 消息后，函数将等待并打印服务器的响应，以确认传输成功。

Now, with the connect_websocket functio, we established a WebSocket connection to a specified URI with authorization headers, using a bearer token for authentication. Upon connection, it first receives an initial response from the server. Then, it reads a Base64-encoded audio file from the specified path and constructs a message with various parameters: a client timestamp, a prompt (asking the server to transcribe the audio), and a control section specifying text response format, server-based voice activity detection (VAD), audio encoding as PCM, voice type, and speed ratio. After sending this JSON message, the function awaits and prints the server’s response to confirm successful transmission.

In [3]:
async def connect_websocket():
    uri = f"wss://open.bigmodel.cn/api/paas/ws/chat"
    extra_headers = {
    "Authorization": "Bearer {0}".format("API KEY")
    }
    async with websockets.connect(uri, extra_headers=extra_headers, ping_interval=120, ping_timeout=200 ) as websocket:
        response = await websocket.recv()
        print(f"服务器响应： {response}")

        audio_base = read_base64_file("vision/data/audio_base64_output.txt")

        message = {
            "client_timestamp": time.time(),
            "system_prompt": "请问这位女士在讲什么？请把你听到的每句话都复述出来。",
            "chunk_type": "append",
            "audio_chunk": audio_base,
            "control": {
                "response_type": "text",
                "vad_config": {
                    "server_vad": True,
                    "finish_time": 60
                },
                "audio_config": {
                    "encoding": "pcm",
                    "voice_type": "NORMAL_FEMALE",
                    "speed_ratio": 1.00
                }
            }
        }

        await websocket.send(json.dumps(message))
        print("消息已发送")

        response = await websocket.recv()
        print(f"服务器响应： {response}")


await connect_websocket()

服务器响应： {"conversation_id":"202411051348056326c7bcf9904cf1","created":1730785685768,"message":{"content":"inited","role":"assistant","type":"event"}}
消息已发送
服务器响应： {"conversation_id":"202411051348056326c7bcf9904cf1","created":1730785713543,"message":{"content":"很抱歉，","role":"assistant","type":"text"},"message_id":"9157393711747323263"}
